In [1]:
#%run download-planet-data.py
# Current line count as of Feb 21, 2020
#       4127 confirmed-planets.csv
#     892 k2-candidate-only-planets.csv
#    1236 k2-confirmed-and-candidate-planets.csv
#     431 k2-confirmed-planets.csv
#   392246 data/k2-targets.csv
#    2421 kepler-candidate-only-planets.csv
#    4724 kepler-confirmed-and-candidate-planets.csv
#    2357 kepler-confirmed-planets.csv
#      42 tess-confirmed-planets.csv


In [6]:
import pandas as pd
import numpy as np

# Read in all confirmed planets
df = pd.read_csv('data/confirmed-planets.csv') 
                             
usecols= [
    'pl_name', 
    'pl_orbsmax', 
    'st_teff', 
    'st_rad', 
    'st_j', 
    'st_mass', 
    'pl_rade', 
    'pl_orbper', 
    'pl_discmethod',
]
df = df[usecols]

# Read in Kepler candidates (cumulative, not including FPs or PCs)
dfKep = pd.read_csv('data/kepler-candidate-only-planets.csv') 
                                    
usecolsKep= [
    'kepoi_name', 
    'kepler_name', 
    'koi_period', 
    'koi_prad', 
    'koi_sma', 
    'koi_steff', 
    'koi_jmag', 
    'koi_smass', 
    'koi_srad',
]
dfKep = dfKep[usecolsKep]


# Read in K2 candidates (not including PCs)
dfK2 = pd.read_csv('data/k2-candidate-only-planets.csv') 
                                
usecolsK2= [
    'epic_name', 
    'pl_name', 
    'pl_orbper', 
    'st_teff', 
    'st_rad', 
    'st_j2', 
    'pl_rade',
]
dfK2 = dfK2[usecolsK2]


# planet name
#name = df.loc[:, 'pl_name'] 
Name = df.pl_name
kep_Name = dfKep.kepoi_name
k2_Name = dfK2.epic_name


# star mass
Mstar = df.st_mass
kep_Mstar = dfKep.koi_smass
# using Rstar as a proxy for K2 for now; no k2CandidatesTable.st_mass and empty values for k2Targets.k2_mass
k2_Mstar = dfK2.st_rad 

# star size in solar units
Rstar = df.st_rad
kep_Rstar = dfKep.koi_srad
k2_Rstar = dfK2.st_rad

# planet size in earth units
Rp = df.pl_rade
kep_Rp = dfKep.koi_prad
k2_Rp = dfK2.pl_rade

# star effective temperature in solar units
Teff = df.st_teff / 5780
kep_Teff = dfKep.koi_steff / 5780
k2_Teff = dfK2.st_teff / 5780

# orbital period in days
Per = df.pl_orbper
kep_Per = dfKep.koi_period
k2_Per = dfK2.pl_orbper 

# semimajor axis in AU
Semimaj = df.pl_orbsmax
kep_Semimaj = dfKep.koi_sma

# calculating semimaj from period (years); no k2CandidatesTable.pl_orbsmax (still need k2_Mstar)
k2_per_yr = df.pl_orbper / 365.25
k2_Semimaj = (k2_Mstar*k2_per_yr**2)**(1/3)

# insolation in earth units
S = Rstar**2 * Teff**4. * (1/Semimaj)**2
kep_S = kep_Rstar**2 * kep_Teff**4. * (1/kep_Semimaj)**2
k2_S = k2_Rstar**2 * k2_Teff**4. * (1/k2_Semimaj)**2

# J-band magnitude
Jmag = df.st_j
kep_Jmag = dfKep.koi_jmag
k2_Jmag = dfK2.st_j2


In [16]:
# Let's make plots now
from bokeh.plotting import figure, output_file, show, ColumnDataSource
from bokeh.io import output_notebook, show
from bokeh.models import HoverTool
from bokeh.resources import CDN
from bokeh.embed import file_html

output_notebook()
# output_file('planets.html')

# Planet Discovery Method Counts
#
# Radial Velocity (793)
# Imaging (49)
# Transit (3165)
# Astrometry (1)
# Microlensing (86)
#   "Timing Variations" includes planets discovered by:
# Eclipse Timing Variations (16)
# Pulsation Timing Variations (2)
# Transit Timing Variations (21)
# Pulsar Timing (7)
# Disk Kinematics (1)
# Orbital Brightness Modulations (6)


# For confirmed planets
discmethod = df.pl_discmethod

source = ColumnDataSource(data=dict(
    x = Per[discmethod == "Transit"],
    y = Rp[discmethod == "Transit"],
    name = Name[discmethod == "Transit"],
    rs = Rstar[discmethod == "Transit"]
))

TOOLTIPS = [
    ("Planet Name", "@name"),
    ("Star radius", "@rs")
]

p = figure(width=500, height=500, title="title", tooltips=TOOLTIPS,
           x_range=(0.1, 1000), y_range=(0.1, 20),
           y_axis_type="log", x_axis_type="log")

p.circle('x', 'y', size=9, color="firebrick", alpha=0.3, source=source,)

p.xaxis.axis_label = "Orbital Period (days)"
p.yaxis.axis_label = "Planet Size (Earth-radii)"


show(p)

html = file_html(p, CDN, "my plot")

Loading BokehJS ...

In [ ]:
html = html.replace('\n\n\n\n<!DOCTYPE html>\n<html lang="en">\n  \n  ',
             '')
html = html.replace('  \n</html>', '')

output_file = '_includes/periodradius.html'
outf = open(output_file,'w')
outf.write(html)
outf.close()

In [ ]:
# for Venus analogs
#df.loc[:,'S'] = S
#dfKep.loc[:,'kep_S'] = kep_S
#dfK2.loc[:,'k2_S'] = k2_S

# small planets with venus insolation, and will want Jmag < 12.  
#idx = (Rp < 2.5) & (S < 7) & (Rstar < 0.4) #& (Jmag < 12)
#kep_idx = (kep_Rp < 2) & (kep_S < 4) & (kep_S > 1) & (kep_Rstar < 0.7) & (kep_Jmag < 13)
#k2_idx = (k2_Rp < 2) & (k2_S < 20) & (k2_S > 1)& (k2_Rstar < 0.4) & (k2_Jmag < 13)

# print tables
#df.loc[idx, ['pl_name', 'pl_rade', 'st_rad', 'st_j', 'S', ]]
#dfKep.loc[kep_idx, ['kepoi_name', 'kepler_name', 'koi_prad', 'koi_srad', 'koi_jmag', 'kep_S']]
#dfK2.loc[k2_idx, ['epic_name', 'pl_name', 'pl_rade', 'st_rad', 'st_j2', 'k2_S']]

